<a href="https://colab.research.google.com/github/Teasotea/BioNER-and-RD/blob/main/create_cid_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re

In [2]:
import nltk
from nltk.tokenize import TreebankWordTokenizer as twt
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Load Data

In [3]:
with open('/content/data0.txt') as f:
      eval_article = f.read()

In [4]:
train_df = pd.read_csv('/content/cdr_dner_train_df.csv')
test_df = pd.read_csv('/content/cdr_dner_test_df.csv')
dev_df = pd.read_csv('/content/cdr_dner_dev_df.csv')


In [5]:
CID_train_df = train_df[train_df['xloc'] == "CID"].reset_index(drop=True)
DNER_train_df = train_df[train_df['xloc'] != "CID"].reset_index(drop=True)

CID_test_df = test_df[test_df['xloc'] == "CID"].reset_index(drop=True)
DNER_test_df = test_df[test_df['xloc'] != "CID"].reset_index(drop=True)

CID_dev_df = dev_df[dev_df['xloc'] == "CID"].reset_index(drop=True)
DNER_dev_df = dev_df[dev_df['xloc'] != "CID"].reset_index(drop=True)

# CID Dataset

In [6]:
CID_train_df["text"] = CID_train_df.apply(lambda x: x['title_source_text']+ ' '+x['source_text'], axis = 1)
CID_test_df["text"] = CID_test_df.apply(lambda x: x['title_source_text']+ ' '+x['source_text'], axis = 1)
CID_dev_df["text"] = CID_dev_df.apply(lambda x: x['title_source_text']+ ' '+x['source_text'], axis = 1)

CID_train_df = CID_train_df.drop(columns = ['entity', 'name_id', 'title_source_text', 'source_text', 'xloc'])
CID_test_df = CID_test_df.drop(columns = ['entity', 'name_id', 'title_source_text', 'source_text', 'xloc'])
CID_dev_df = CID_dev_df.drop(columns = ['entity', 'name_id', 'title_source_text', 'source_text', 'xloc'])


In [7]:
CID_train_df = CID_train_df[CID_train_df['name'].str.startswith('D') & CID_train_df['yloc'].str.startswith('D')].rename(columns = {'yloc':'w1', 'name':'w2'})
CID_test_df = CID_test_df[CID_test_df['name'].str.startswith('D') & CID_test_df['yloc'].str.startswith('D')].rename(columns = {'yloc':'w1', 'name':'w2'})
CID_dev_df = CID_dev_df[CID_dev_df['name'].str.startswith('D') & CID_dev_df['yloc'].str.startswith('D')].rename(columns = {'yloc':'w1', 'name':'w2'})

In [8]:
CID_train_df.head()

,text_id,w1,w2,text
0,354896,D008012,D006323,Lidocaine-induced cardiac asystole. Intravenou...
1,435349,D013390,D005207,Suxamethonium infusion rate and observed fasci...
2,603022,D012601,D062787,"Galanthamine hydrobromide, a longer acting ant..."
3,1378968,D008094,D006973,Effects of uninephrectomy and high protein fee...
4,1378968,D008094,D011507,Effects of uninephrectomy and high protein fee...


In [54]:
def prepr_df(df, df2):
  nameid_df = df[['name','entity', 'name_id']]
  nameid_df = nameid_df[nameid_df['name_id'].str.contains('D', na=False)]
  nameid_df["name_entity"] = nameid_df.apply(lambda x: {x["name"]: x["entity"]},axis=1)
  nameid_df = nameid_df.drop(columns=['name', 'entity'])
  nameid_df.set_index('name_id', inplace=True)
  final_df = df2.replace({"w1": nameid_df.squeeze().to_dict(), "w2": nameid_df.squeeze().to_dict()})
  return final_df

In [55]:
train_cid= prepr_df(DNER_train_df, CID_train_df)
test_cid = prepr_df(DNER_test_df, CID_test_df)
dev_cid = prepr_df(DNER_dev_df, CID_dev_df)

In [56]:
train_cid.head()

,text_id,w1,w2,text
0,354896,{'lidocaine': 'Chemical'},{'cardiac arrest': 'Disease'},Lidocaine-induced cardiac asystole. Intravenou...
1,435349,{'Suxamethonium': 'Chemical'},{'fasciculation': 'Disease'},Suxamethonium infusion rate and observed fasci...
2,603022,{'scopolamine': 'Chemical'},{'overdose': 'Disease'},"Galanthamine hydrobromide, a longer acting ant..."
3,1378968,{'lithium': 'Chemical'},{'hypertension': 'Disease'},Effects of uninephrectomy and high protein fee...
4,1378968,{'lithium': 'Chemical'},{'proteinuria': 'Disease'},Effects of uninephrectomy and high protein fee...


In [57]:
train_cid.to_csv('train_cid.csv', index=False)
test_cid.to_csv('test_cid.csv', index=False)
dev_cid.to_csv('dev_cid.csv', index = False)